In [1]:
import pandas as pd
from pandas import HDFStore
import numpy as np
import h5py

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def onehot_encode(seq):
    '''
    one-hot encode a DNA sequence that contains alphabet of ["A", "C", "G", "T", "N"]
    A -> [1, 0, 0, 0]
    C -> [0, 1, 0, 0]
    G -> [0, 0, 1, 0]
    T -> [0, 0, 0, 1]
    N -> [0, 0, 0, 0]
    '''
    
    seq_list = list(seq)
    label_encoder = LabelEncoder()
    label_encoder.fit(np.array(['A','C','G','T','N']))
    integer_encoded = label_encoder.transform(seq_list)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int, categories=[range(5)])
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return np.delete(onehot_encoded, -2, 1)




/home/qihzhong/anaconda3/envs/bio-ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load and preview rna expression data

In [2]:
pc = pd.read_table("data/57epigenomes.RPKM.pc", index_col=False)
nc = pd.read_table("data/57epigenomes.RPKM.nc", index_col=False)

In [3]:
print(pc.shape)
pc.head()

(19795, 58)


,gene_id,E000,E003,E004,E005,E006,E007,E011,E012,E013,...,E114,E116,E117,E118,E119,E120,E122,E123,E127,E128
0,ENSG00000000003,23.265,43.985,37.413,29.459,21.864,55.649,52.940,71.629,61.292,...,37.989,0.038,42.639,49.983,11.554,11.847,43.723,0.267,13.758,15.818
1,ENSG00000000005,0.872,1.642,6.498,0.000,0.157,0.003,0.115,0.087,0.055,...,0.000,0.000,0.000,0.000,0.000,0.018,0.000,0.006,0.000,0.000
2,ENSG00000000419,55.208,35.259,58.308,48.208,37.477,45.923,44.959,40.438,41.970,...,52.215,79.197,107.098,62.811,42.386,54.869,16.652,73.719,56.578,56.371
3,ENSG00000000457,3.237,2.596,2.345,8.775,2.723,3.700,3.912,5.011,4.158,...,4.829,11.082,8.814,2.646,2.483,2.527,2.549,7.651,4.967,3.714
4,ENSG00000000460,7.299,6.649,7.838,7.324,0.830,5.354,5.940,5.704,6.213,...,8.001,13.743,25.369,3.373,4.646,2.179,4.099,22.103,3.290,2.491


# Concat protein coding genes and non protein coding genes

In [4]:
all_genes = pd.concat([pc, nc], ignore_index=True)
all_genes.set_index("gene_id", inplace=True)

# Read dna sequence flanking TSS

In [5]:
from Bio import SeqIO
records = list(SeqIO.parse("data/input_sequence_2000bps.fa", "fasta"))



In [6]:
print("The fa file has", len(records) ,"sequence")

# display the first element in the list
print(records[0])  # first record


The fa file has 49737 sequence
ID: ENSG00000223972
Name: ENSG00000223972
Description: ENSG00000223972
Number of features: 0
Seq('CACATGCTACCGCGTCCAGGGGTGGAGGCGTGGCGCAGGCGCAGAGAGGCGCAC...GCC', SingleLetterAlphabet())


In [7]:
# df = pd.DataFrame(columns=["seq"], index=["gene_id"])

In [8]:
import time
tic = time.clock()

all_genes["seq"] = None

for i in range(len(records)):
# for i in range(1000):
    if (records[i].name in all_genes.index):
        all_genes.loc[records[i].name, "seq"] = str(records[i].seq)
        
toc = time.clock()
toc - tic

113.819829

In [9]:
all_genes

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E116,E117,E118,E119,E120,E122,E123,E127,E128,seq
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,23.265,43.985,37.413,29.459,21.864,55.649,52.940,71.629,61.292,44.280,...,0.038,42.639,49.983,11.554,11.847,43.723,0.267,13.758,15.818,AGGTCAGGAGATCGAGACCATCCTGTCTAACACGGTGAAACCCCGT...
ENSG00000000005,0.872,1.642,6.498,0.000,0.157,0.003,0.115,0.087,0.055,1.577,...,0.000,0.000,0.000,0.000,0.018,0.000,0.006,0.000,0.000,TATGAATGGGACCTTATCCAATATAACTGATGTTCTTATAATGAAA...
ENSG00000000419,55.208,35.259,58.308,48.208,37.477,45.923,44.959,40.438,41.970,51.515,...,79.197,107.098,62.811,42.386,54.869,16.652,73.719,56.578,56.371,TTGGTCACTGTCTCCGCTGGGGGTGGTTGGGGGAATATGCAGCGAT...
ENSG00000000457,3.237,2.596,2.345,8.775,2.723,3.700,3.912,5.011,4.158,3.292,...,11.082,8.814,2.646,2.483,2.527,2.549,7.651,4.967,3.714,TGTTTTTCAGTCCAAACCTATTCCTGAATCTGTGTAGCCATCCCTT...
ENSG00000000460,7.299,6.649,7.838,7.324,0.830,5.354,5.940,5.704,6.213,7.551,...,13.743,25.369,3.373,4.646,2.179,4.099,22.103,3.290,2.491,TAACCAAAACAGCATGGTACTGGTACTGAGAGGTGACAGCGTGCTG...
ENSG00000000938,0.052,0.211,0.059,0.009,0.012,0.013,0.012,0.012,0.013,0.094,...,98.303,0.016,0.008,0.052,0.001,0.137,0.016,0.047,0.000,TACAGGCGCCCGCCACCACGCCCGGCTAATTTTTTGATTTTTAGTA...
ENSG00000000971,3.000,0.000,0.003,0.009,41.909,0.002,0.011,0.018,0.121,0.004,...,0.002,124.801,0.007,0.419,39.724,8.464,41.504,1.348,32.725,TATCCATTACACCTAAACATAAATTATGTCCCAGTGTTTCACTACC...
ENSG00000001036,58.371,49.337,25.066,25.679,86.255,21.922,33.225,22.268,37.503,35.061,...,18.191,53.641,74.928,12.045,45.419,27.254,59.833,11.164,82.445,AAGCTTGCAGGTTTCTTAAACATTCATTCCACTGGTTTCACCTGTC...
ENSG00000001084,7.077,6.966,10.110,4.609,5.870,6.440,5.431,6.838,3.018,9.897,...,4.571,10.435,10.065,18.571,1.547,2.588,2.750,22.945,5.273,ACACTTTCTGGGTAGGTACATGTATGATCTAGTTGCCTTCACTTTT...


In [10]:
common_genes = all_genes[all_genes["seq"].values != None]


In [11]:
del records
del all_genes

In [25]:
common_genes

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E116,E117,E118,E119,E120,E122,E123,E127,E128,seq
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,23.265,43.985,37.413,29.459,21.864,55.649,52.940,71.629,61.292,44.280,...,0.038,42.639,49.983,11.554,11.847,43.723,0.267,13.758,15.818,AGGTCAGGAGATCGAGACCATCCTGTCTAACACGGTGAAACCCCGT...
ENSG00000000005,0.872,1.642,6.498,0.000,0.157,0.003,0.115,0.087,0.055,1.577,...,0.000,0.000,0.000,0.000,0.018,0.000,0.006,0.000,0.000,TATGAATGGGACCTTATCCAATATAACTGATGTTCTTATAATGAAA...
ENSG00000000419,55.208,35.259,58.308,48.208,37.477,45.923,44.959,40.438,41.970,51.515,...,79.197,107.098,62.811,42.386,54.869,16.652,73.719,56.578,56.371,TTGGTCACTGTCTCCGCTGGGGGTGGTTGGGGGAATATGCAGCGAT...
ENSG00000000457,3.237,2.596,2.345,8.775,2.723,3.700,3.912,5.011,4.158,3.292,...,11.082,8.814,2.646,2.483,2.527,2.549,7.651,4.967,3.714,TGTTTTTCAGTCCAAACCTATTCCTGAATCTGTGTAGCCATCCCTT...
ENSG00000000460,7.299,6.649,7.838,7.324,0.830,5.354,5.940,5.704,6.213,7.551,...,13.743,25.369,3.373,4.646,2.179,4.099,22.103,3.290,2.491,TAACCAAAACAGCATGGTACTGGTACTGAGAGGTGACAGCGTGCTG...
ENSG00000000938,0.052,0.211,0.059,0.009,0.012,0.013,0.012,0.012,0.013,0.094,...,98.303,0.016,0.008,0.052,0.001,0.137,0.016,0.047,0.000,TACAGGCGCCCGCCACCACGCCCGGCTAATTTTTTGATTTTTAGTA...
ENSG00000000971,3.000,0.000,0.003,0.009,41.909,0.002,0.011,0.018,0.121,0.004,...,0.002,124.801,0.007,0.419,39.724,8.464,41.504,1.348,32.725,TATCCATTACACCTAAACATAAATTATGTCCCAGTGTTTCACTACC...
ENSG00000001036,58.371,49.337,25.066,25.679,86.255,21.922,33.225,22.268,37.503,35.061,...,18.191,53.641,74.928,12.045,45.419,27.254,59.833,11.164,82.445,AAGCTTGCAGGTTTCTTAAACATTCATTCCACTGGTTTCACCTGTC...
ENSG00000001084,7.077,6.966,10.110,4.609,5.870,6.440,5.431,6.838,3.018,9.897,...,4.571,10.435,10.065,18.571,1.547,2.588,2.750,22.945,5.273,ACACTTTCTGGGTAGGTACATGTATGATCTAGTTGCCTTCACTTTT...


In [28]:
tic = time.clock()


# sample_size = len(common_genes)
# for i in range(sample_size):
#     common_genes["seq"][i] = onehot_encode(common_genes["seq"][i])


common_genes = pd.concat([common_genes.drop('seq', axis=1), common_genes['seq'].apply(onehot_encode)], axis=1)
    
toc = time.clock()
toc - tic

42.67150800000002

In [29]:
common_genes

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E116,E117,E118,E119,E120,E122,E123,E127,E128,seq
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,23.265,43.985,37.413,29.459,21.864,55.649,52.940,71.629,61.292,44.280,...,0.038,42.639,49.983,11.554,11.847,43.723,0.267,13.758,15.818,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
ENSG00000000005,0.872,1.642,6.498,0.000,0.157,0.003,0.115,0.087,0.055,1.577,...,0.000,0.000,0.000,0.000,0.018,0.000,0.006,0.000,0.000,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
ENSG00000000419,55.208,35.259,58.308,48.208,37.477,45.923,44.959,40.438,41.970,51.515,...,79.197,107.098,62.811,42.386,54.869,16.652,73.719,56.578,56.371,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [0,..."
ENSG00000000457,3.237,2.596,2.345,8.775,2.723,3.700,3.912,5.011,4.158,3.292,...,11.082,8.814,2.646,2.483,2.527,2.549,7.651,4.967,3.714,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
ENSG00000000460,7.299,6.649,7.838,7.324,0.830,5.354,5.940,5.704,6.213,7.551,...,13.743,25.369,3.373,4.646,2.179,4.099,22.103,3.290,2.491,"[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [0,..."
ENSG00000000938,0.052,0.211,0.059,0.009,0.012,0.013,0.012,0.012,0.013,0.094,...,98.303,0.016,0.008,0.052,0.001,0.137,0.016,0.047,0.000,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0], [1,..."
ENSG00000000971,3.000,0.000,0.003,0.009,41.909,0.002,0.011,0.018,0.121,0.004,...,0.002,124.801,0.007,0.419,39.724,8.464,41.504,1.348,32.725,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
ENSG00000001036,58.371,49.337,25.066,25.679,86.255,21.922,33.225,22.268,37.503,35.061,...,18.191,53.641,74.928,12.045,45.419,27.254,59.833,11.164,82.445,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0,..."
ENSG00000001084,7.077,6.966,10.110,4.609,5.870,6.440,5.431,6.838,3.018,9.897,...,4.571,10.435,10.065,18.571,1.547,2.588,2.750,22.945,5.273,"[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,..."


In [30]:
common_genes.shape

(42438, 58)

In [44]:
import pickle
import os.path

file_path = "common_genes.pkl"
max_bytes = 2**31 - 1


## write
bytes_out = pickle.dumps(common_genes)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])


In [45]:
## read
bytes_in = bytearray(0)
input_size = os.path.getsize(file_path)
with open(file_path, 'rb') as f_in:
    for _ in range(0, input_size, max_bytes):
        bytes_in += f_in.read(max_bytes)
data2 = pickle.loads(bytes_in)

(42438, 58)

In [16]:
sample_size = len(common_genes)

# hdf = HDFStore("samples" + str(sample_size) + ".h5")
# hdf.put("samples" + str(sample_size), common_genes[:sample_size])
# hdf.close()

/anaconda3/envs/LSTM/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['seq']]

  if self.run_code(code, result):


OverflowError: value too large to convert to int